# DIMS.FARM REST API example

In [1]:
import os
import json

import pandas as pd
from requests import HTTPError

from watobs.datafarm import DatafarmRepository

api_key = os.getenv("DATAFARM_API_KEY")
assert api_key is not None


### Connect to DIMS.FARM REST API

In [2]:
dfr = DatafarmRepository(api_key)
dfr.connect()

### List available time series

In [3]:
time_series_list = dfr.list_time_series()
time_series_list.tail()

,ID,EntityID,Touched,IDName,IDDescription,LocationID,TimeSeriesParameterID,TimeSeriesMediaID,TimeSeriesUnitID,TimeSeriesDataTypeID,TimeSeriesDatasourceID,TimeSeriesTypeID,TimeSeriesStatusID,TimeSeriesDataProviderID,TimeSeriesDataProviderArgument,DataExpectedCount,DataArchiveAfter,DataDeleteAfter
GUID,,,,,,,,,,,,,,,,,,
{35D46EDD-C352-11ED-B2F7-1831BF2DC749},2087,TNWB_wind_RVO-FUGRO_unfiltered_WS-120,2023-03-20 18:25:13,TNWB_wind_RVO-FUGRO_unfiltered_WS-120,Wind speed,TNWB,WS,wind,m/s,Unknown,,RVO-FUGRO,unfiltered,,,NaN,NaN,NaN
{35D46EDF-C352-11ED-B2F7-1831BF2DC749},2088,TNWB_wind_RVO-FUGRO_unfiltered_WS-130,2023-03-20 18:25:14,TNWB_wind_RVO-FUGRO_unfiltered_WS-130,Wind speed,TNWB,WS,wind,m/s,Unknown,,RVO-FUGRO,unfiltered,,,NaN,NaN,NaN
{35D46EE1-C352-11ED-B2F7-1831BF2DC749},2089,WHI_waves_CMEMS_unfiltered_Hm0,2023-03-20 18:25:14,WHI_waves_CMEMS_unfiltered_Hm0,Spectral significant wave height,WHI,Hm0,waves,m,Unknown,,CMEMS,unfiltered,,,NaN,NaN,NaN
{3C93FFDB-C352-11ED-B2F7-1831BF2DC749},2090,WHI_waves_CMEMS_unfiltered_Tz,2023-03-20 18:25:15,WHI_waves_CMEMS_unfiltered_Tz,Average zero crossing wave period (Tz),WHI,Tz,waves,s,Unknown,,CMEMS,unfiltered,,,NaN,NaN,NaN
{7B4D3518-EE55-11ED-B2FB-1831BF2DC749},2091,testapi.insert,2023-05-25 12:21:41,testapi.insert,,,,,,Unknown,,,,,,NaN,NaN,NaN


In [4]:
dfr.get_statistics(time_series_list['EntityID'].iloc[-10:])

,First,Last,DailyCount
Name,,,
TNWB_wind_RVO-FUGRO_unfiltered_WS-70,2019-06-19,2021-02-16,144
TNWB_wind_RVO-FUGRO_unfiltered_WS-80,2019-06-19,2021-02-16,144
TNWB_wind_RVO-FUGRO_unfiltered_WS-90,2019-06-19,2021-02-16,144
TNWB_wind_RVO-FUGRO_unfiltered_WS-100,2019-06-19,2021-02-16,144
TNWB_wind_RVO-FUGRO_unfiltered_WS-110,2019-06-19,2021-02-16,144
TNWB_wind_RVO-FUGRO_unfiltered_WS-120,2019-06-19,2021-02-16,144
TNWB_wind_RVO-FUGRO_unfiltered_WS-130,2019-06-19,2021-02-16,144
WHI_waves_CMEMS_unfiltered_Hm0,2011-05-19,2022-10-01,48
WHI_waves_CMEMS_unfiltered_Tz,2011-05-19,2022-10-01,48


### Get time series data

In [5]:
data = dfr.get_data(
    time_series_id="testapi.insert",
    limit=5,
    ascending=True
)
data

,Data,QualityTxt
RefDateTimeRef,,
2012-11-29 15:36:29,0.012895,ok
2014-10-15 15:36:29,0.196728,ok
2022-09-28 23:56:50,637.000000,ok
2022-09-28 23:56:50,637.000000,ok
2022-09-29 00:26:43,637.000000,ok


### Delete data in a given range:

In [6]:
start, end = data.index[0], data.index[2]
print(f"Deleting data in range [{start}, {end})")
res = dfr.delete_data("testapi.insert", start=start, end=end)
res.json()

Deleting data in range [2012-11-29 15:36:29, 2022-09-28 23:56:50)


{'result': 'delete range successful'}

In [7]:
timestamps = data.index[:3]
print(f"Deleting data for timestamps {timestamps}")
res = dfr.delete_data("testapi.insert", timestamps=timestamps)
res.json()

Deleting data for timestamps DatetimeIndex(['2012-11-29 15:36:29', '2014-10-15 15:36:29',
               '2022-09-28 23:56:50'],
              dtype='datetime64[ns]', name='RefDateTimeRef', freq=None)


{'result': '3 items deleted'}

In [8]:
res.json()

{'result': '3 items deleted'}

Check that the data has been deleted:

In [9]:
data = dfr.get_data(
    time_series_id="testapi.insert",
    limit=10,
    ascending=True
)
data

,Data,QualityTxt
RefDateTimeRef,,
2022-09-28 23:56:50,637.0,ok
2022-09-28 23:56:50,637.0,ok
2022-09-29 00:26:43,637.0,ok
2022-09-29 00:26:48,637.0,ok
2022-09-29 00:26:48,637.0,ok
2022-09-29 00:56:40,637.0,ok
2022-09-29 00:56:50,637.0,ok
2022-09-29 00:56:50,637.0,ok
2022-09-29 01:26:43,637.0,ok


### Insert time series data

Here we prepare some data to insert in form of a dataframe.

In [10]:
import random

random_date = lambda: pd.Timestamp.now() - pd.Timedelta(days=3000 + random.randint(1, 1000))
rows = 5
new_data = pd.DataFrame({
    "TimeStamp": [random_date() for _ in range(rows)],
    "Data": [random.random() for _ in range(rows)],
    "Quality": ["ok"] * rows,
})
new_data.loc[0, "Data"] = None
new_data


,TimeStamp,Data,Quality
0,2013-08-29 15:41:13.236187,NaN,ok
1,2015-03-16 15:41:13.236407,0.982861,ok
2,2014-05-09 15:41:13.236443,0.120583,ok
3,2012-07-04 15:41:13.236471,0.631634,ok
4,2013-10-30 15:41:13.236496,0.326983,ok


To insert the data we use the `insert_data` method:

In [11]:
try:
    res = dfr.insert_data("testapi.insert", new_data, bulk_insert=True)
except HTTPError as e:
    print(e)

In [12]:
res.json()

{'ItemsWritten': 5}

We can peek at the json body for the call to the API. The NaN value is represented as {"N": 1, "V": 0}.

In [13]:
json.loads(res.request.body)

{'BulkInsert': True,
 'TimeSeriesName': 'testapi.insert',
 'TimeStamp': ['2013-08-29T15:41:13.236187Z',
  '2015-03-16T15:41:13.236407Z',
  '2014-05-09T15:41:13.236443Z',
  '2012-07-04T15:41:13.236471Z',
  '2013-10-30T15:41:13.236496Z'],
 'Data': [{'N': 1, 'V': 0.0},
  {'N': 0, 'V': 0.9828613527808476},
  {'N': 0, 'V': 0.12058282287271727},
  {'N': 0, 'V': 0.6316336466896517},
  {'N': 0, 'V': 0.3269827625033659}],
 'QualityLevel': [0, 0, 0, 0, 0]}

### Using the context manager

In [16]:

with DatafarmRepository(api_key) as dfr:
    data = dfr.get_data(
        time_series_id="testapi.insert",
        limit=2
    )
data

,Data,QualityTxt
RefDateTimeRef,,
2012-07-04 15:41:13,0.631634,ok
2013-08-29 15:41:13,NaN,ok
